## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
sns.set_style('darkgrid')

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [2]:
df = pd.read_csv('./QueryResults.csv')
df.sample()

FileNotFoundError: [Errno 2] No such file or directory: './QueryResults.csv'

In [ ]:
df.rename(columns = {'Unnamed: 2': 'Posts', 'TagName': 'Tag', 'm': 'Date'}, inplace = True)
df.sample()

,Date,Tag,Posts
1051,2014-12-01 00:00:00,swift,2501


**Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [ ]:
pd.concat([df.head(), df.tail()])

,Date,Tag,Posts
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,javascript,162
3,2008-08-01 00:00:00,c,85
4,2008-08-01 00:00:00,python,124
1986,2020-07-01 00:00:00,r,5694
1987,2020-07-01 00:00:00,go,743
1988,2020-07-01 00:00:00,ruby,775
1989,2020-07-01 00:00:00,perl,182
1990,2020-07-01 00:00:00,swift,3607


**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [ ]:
df.shape

(1991, 3)

In [ ]:
df.ndim

2

**Challenge**: Count the number of entries in each column of the dataframe

In [ ]:
df.count()

Date     1991
Tag      1991
Posts    1991
dtype: int64

**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [ ]:
df.groupby('Tag').sum().sort_values('Posts').tail(1).reset_index()[['Tag', 'Posts']]

,Tag,Posts
0,javascript,2056510


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [ ]:
df.groupby('Tag').count()

,Date,Posts
Tag,,
assembly,144,144
c,144,144
c#,145,145
c++,144,144
delphi,144,144
go,129,129
java,144,144
javascript,144,144
perl,144,144


## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [ ]:
type(df['Date'].iloc[0])

str

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
type(df['Date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

## Data Manipulation



In [ ]:
df = df.pivot_table('Posts', 'Date', 'Tag', fill_value = 0)

**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

In [ ]:
df.ndim

2

In [ ]:
df.shape

(145, 14)

In [ ]:
df.columns

Index(['assembly', 'c', 'c#', 'c++', 'delphi', 'go', 'java', 'javascript',
       'perl', 'php', 'python', 'r', 'ruby', 'swift'],
      dtype='object', name='Tag')

In [ ]:
df.head()

Tag,assembly,c,c#,c++,delphi,go,java,javascript,perl,php,python,r,ruby,swift
Date,,,,,,,,,,,,,,
2008-07-01,0,0,3,0,0,0,0,0,0,0,0,0,0,0
2008-08-01,8,85,511,164,14,0,222,162,28,161,124,0,73,0
2008-09-01,28,321,1649,755,105,0,1137,640,131,482,542,6,290,0
2008-10-01,15,303,1989,811,112,0,1153,725,127,617,510,0,249,0
2008-11-01,17,259,1730,735,141,0,958,579,97,504,452,1,160,0


**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

In [ ]:
df.count()

Tag
assembly      145
c             145
c#            145
c++           145
delphi        145
go            145
java          145
javascript    145
perl          145
php           145
python        145
r             145
ruby          145
swift         145
dtype: int64

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

In [ ]:
px.line(df['java'])

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

In [ ]:
px.line(df[['java', 'python']])

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 

In [ ]:
px.line(df.rolling(6).mean())

In [ ]:
px.line(df.rolling(12).mean())